The data provided by James Ko (https://www.kaggle.com/jameslko/gun-violence-data) and extracted from the Gun Violence Archive (http://www.gunviolencearchive.org) contains 239,677 recorded incidents from January 2013 to March 2018, presented in increasing chronological order. The raw dataset contains 29 features which exhibit information about the time, location, and participants of the event, as well as sources to confirm its veridicality. The features presented in the dataset, together with their corresponding descriptions, are as follows:

1. **address**: address of the location of the attack
2. **city_or_county**: city/county of the attack
3. **congressional_district**: the congressional district the attack occured under
4. **date**: date in format "YYYY-MM-DD"
5. **gun_stolen**: boolean values indicating whether each gun used in the attack was stolen, stored in a dictionary format
6. **gun_type**: type of each gun used in the attack in a dictionary format
7. **incident_characteristics**: characteristics of the incident, in a list format	
8. **incident_id**: GVA unique identification number of incident
9. **incident_url**: GVA URL corresponding to the incident
10. **incident_url_fields_missing**: boolean indicating whether the incident URL field is missing
11. **latitude**: latitude of the location of the attach
12. **location_description**: characteristics of the location of the attack
13. **longitude**: longitude of the location of the attach
14. **n_guns_involved**: number of guns involved in the attack
15. **n_injured**: total number of people injured in the attack	
16. **n_killed**: total number of people killed in the attack
17. **notes**: miscellaneous notes about the incident
18. **participant_age**: the ages of the participants involved in the attack
19. **participant_age_group**: the age groups the participants involved in the attack fall within; possible values are:
    1. Child 0-11
    2. Teen 12-17
    3. Adult 18+ 
20. **participant_gender**: the genders of the participants involved in the attack
21. **participant_name**: the names of the participants involved in the attack
22. **participant_relationship**: the relationships between the suspect and the victims of the attack; possible values are: 
    1. 'Home Invasion - Perp Does Not Know Victim' or 'Home Invasion - Perp Knows Victim'
    2. 'Neighbor', 'Co-worker'
    3. 'Drive by - Random victims'
    4. 'Family', 'Friends', 'Aquaintance'
    5. 'Significant others - current or former'
    6. 'Gang vs Gang'
    7. 'Armed Robbery'
    8. 'Mass shooting - Random victims' or 'Mass shooting - Perp Knows Victims'
23. **participant_status**: extent of harm done to the participants of the attack; possible values are:
    1. Killed
    2. Injured
    3. Unharmed
    4. Arrested
24. **participant_type**: type of the participant; possible values are: 
    1. Victim
    2. Subject-suspect
25. **source_url**: URL used as a primary source for the data
26. **sources**: reporting sources
27. **state**: state of the crime
28. **state_house_district**: voting house district
29. **state_senate_district**: territorial district from which a senator to a state legislature is elected

The data has been collected using web scraping techniques, and excludes the Las Vegas incident from 2017, the deadliest mass shooting committed by an individual in the history of the United States. In this notebook the respective incident will be merged to the raw dataset. Based on the original scraping code (https://github.com/jamesqo/gun-violence-data), incident data has been collected up until September 2019. At the time of this writing, however, data concerning the guns used and the participants involved in the shootings between March 2018 and September 2019 has not been obtained.

In [109]:
import pandas as pd
import numpy as np
import Levenshtein as lev
import random
import re
from functools import reduce
import itertools
from scipy.stats import skew
import missingno as msno
from datetime import datetime as dt
import data_exploration as de

In [110]:
import warnings
warnings.filterwarnings("ignore")

In [111]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [112]:
incidents = pd.read_csv("stage3_data/gun-violence-data_01-2013_03-2018.csv")

As notified by James Ko, the Las Vegas mass shooting incident from 2017 is missing from the original dataset. A separate .csv file is then provided and merged to the raw data.

In [113]:
lv_incident = pd.read_csv("stage3_data/las-vegas-1-10-2017.csv")
incidents = pd.concat([incidents, lv_incident])

In [114]:
incidents.head(3)

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0


# Data formatting

In [115]:
# incidents.set_index("incident_id", inplace=True)
incidents = incidents.drop(["incident_url_fields_missing", "location_description", "notes", "incident_url", "source_url", "sources"], axis=1)

In [116]:
incidents["date"] = pd.to_datetime(incidents["date"])
incidents = incidents.sort_values(by=["date"])

Each individual incident before March 2018 contains information about all participants in it: **participant_name**, **participant_age**, **participant_age_group**, **participant_gender**, **participant_relationship**, **participant_status**, and **participant_type**. In the event that multiple participants took part in one incident, the raw data comprises the features of multiple participants in a string dictionary format: \\

\begin{gather}
{participant_1}:{value_1} || \ {participant_2}:{value_2} || \ \cdots
\end{gather}

Likewise, features of the guns involved in the incident such as **gun_stolen** and **gun_type** may also accont for multiple guns used in an incident by using the same type of formatting: \\

\begin{gather}
{gun_1}:{value_1} || \ {gun_2}:{value_2} || \ \cdots
\end{gather}

Lastly, the **incident_characteristic** column has a similar format, the only difference being that it requires a list string instead of a dictionary: \\

\begin{gather}
{characteristic_1} || \ {characteristic_2} || \ \cdots
\end{gather}

The next section converts these formats to the standard Python dictionary or list format.

In [117]:
def split_into_dict(string):
    if pd.isnull(string):
        return np.nan
    str_dict = dict()
    # The dataset contains instances separated by | or ||, and : or :: respectively
    # Thus we have to consider both cases when working on those strings
    str_list = '#$%'.join('#$%'.join(string.split('||')).split('|')).split('#$%')
    for index in str_list:
        # If indices and values are split by '::'
        i, sep, v = index.partition('::')
        if sep and v:
            str_dict[int(i)] = v
        # If indices and values are split by ':'
        else:
            i, _, v = index.partition(':')
            str_dict[int(i)] = v
    return str_dict

In [118]:
def convert_vals_to_int(str_dict):
    if pd.isnull(str_dict):
        return np.nan
    for key in str_dict:
        str_dict[key] = int(str_dict[key])
    return str_dict

In [119]:
def split_into_list(string):
    if pd.isnull(string):
        return np.nan
    # Split the string by '|' or '||', depending on the format
    return '#$%'.join('#$%'.join(string.split('||')).split('|')).split('#$%')

In [120]:
for col in ['gun_stolen', 'gun_type', 'participant_name', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_relationship', 'participant_status', 'participant_type']:
    incidents[col] = incidents[col].apply(split_into_dict)
incidents['participant_age'] = incidents['participant_age'].apply(convert_vals_to_int)
incidents['incident_characteristics'] = incidents['incident_characteristics'].apply(split_into_list)

The next task is to split the raw dataset in 3: one containing all **incidents** with incident characteristics as features, i.e. date, state, city/county etc., one containing all **guns** involved in the incidents, with type and origin as features, and lastly a dataset with all **participants** and their details (name, age, status etc.)

In [121]:
df_guns = incidents[["gun_type", "gun_stolen", "incident_id"]].copy()

In [122]:
# Create a NumPy array to store gun information, then converting it to DataFrame for performance improvement
guns_array = np.empty(shape=(int(incidents["n_guns_involved"].sum()), len(df_guns.columns)), dtype=object)
rows = df_guns.values
array_i = 0
for df_i in range(df_guns.shape[0]):
    if pd.notnull(rows[df_i][df_guns.columns.get_loc("gun_type")]):
        for k, v in rows[df_i][df_guns.columns.get_loc("gun_type")].items():
            if pd.notnull(rows[df_i][df_guns.columns.get_loc("gun_stolen")]):
                if k in rows[df_i][df_guns.columns.get_loc("gun_stolen")].keys():
                    guns_array[array_i][df_guns.columns.get_loc("gun_stolen")] = rows[df_i][df_guns.columns.get_loc("gun_stolen")][k]
            guns_array[array_i][df_guns.columns.get_loc("gun_type")] = v
            guns_array[array_i][df_guns.columns.get_loc("incident_id")] = rows[df_i][df_guns.columns.get_loc("incident_id")]
            array_i += 1
guns = pd.DataFrame(guns_array, columns=["type", "stolen", "incident_id"])

In [123]:
df_participants = incidents[["participant_name", "participant_age", "participant_age_group", "participant_gender", "participant_relationship", "participant_status", "participant_type", "incident_id"]].copy()

In [124]:
n_participants = 0
for row in incidents.values:
    if pd.notnull(row[incidents.columns.get_loc("participant_type")]):
        n_participants += len(row[incidents.columns.get_loc("participant_type")])

In [125]:
participants_array = np.empty(shape=(n_participants, len(df_participants.columns)), dtype=object)
rows = df_participants.values
array_i = 0
for df_i in range(df_participants.shape[0]):
    if pd.notnull(rows[df_i][df_participants.columns.get_loc("participant_type")]):
        for k, v in rows[df_i][df_participants.columns.get_loc("participant_type")].items():
            for col in ["participant_name", "participant_age"]:
                if pd.notnull(rows[df_i][df_participants.columns.get_loc(col)]):
                    if k in rows[df_i][df_participants.columns.get_loc(col)].keys():
                        participants_array[array_i][df_participants.columns.get_loc(col)] = rows[df_i][df_participants.columns.get_loc(col)][k]
            if pd.notnull(rows[df_i][df_participants.columns.get_loc("participant_age_group")]):
                if k in rows[df_i][df_participants.columns.get_loc("participant_age_group")].keys():
                    # Some age_group values of the same value have different formats, i.e. "Teen (12-17)" and "Teen 12-17"
                    if rows[df_i][df_participants.columns.get_loc("participant_age_group")][k] == "Teen (12-17)":
                        participants_array[array_i][df_participants.columns.get_loc("participant_age_group")] = "Teen 12-17"
                    else:
                        participants_array[array_i][df_participants.columns.get_loc("participant_age_group")] = rows[df_i][df_participants.columns.get_loc("participant_age_group")][k]
            for col in ["participant_gender", "participant_relationship", "participant_status"]:
                if pd.notnull(rows[df_i][df_participants.columns.get_loc(col)]):
                    if k in rows[df_i][df_participants.columns.get_loc(col)].keys():
                        participants_array[array_i][df_participants.columns.get_loc(col)] = rows[df_i][df_participants.columns.get_loc(col)][k]
            participants_array[array_i][df_participants.columns.get_loc("participant_type")] = v
            participants_array[array_i][df_participants.columns.get_loc("incident_id")] = rows[df_i][df_participants.columns.get_loc("incident_id")]
            array_i += 1     
participants = pd.DataFrame(participants_array, columns=["name", "age", "age_group", "gender", "relationship", "status", "type", "incident_id"])

In [126]:
incidents = incidents[["incident_id", "date", "state", "city_or_county", "address", "n_killed", "n_injured"]]

In the following, the **incidents** dataset will be updated to include the data available on the GVA website up until September 2019.

In [133]:
stage1 = []
for year in range(2013, 2019):
    for month in range(1, 13):
        stage1.append(pd.read_csv("stage1_data/stage1.{0:02d}.{1:04d}.csv".format(month, year)))
for month in range(1, 9):
    stage1.append(pd.read_csv("stage1_data/stage1.{0:02d}.2019.csv".format(month)))

In [134]:
stage1_incidents = pd.concat(stage1)
# Order rows wrt date
stage1_incidents["date"] = pd.to_datetime(stage1_incidents["date"], infer_datetime_format=True)
stage1_incidents = stage1_incidents.sort_values(by=["date"])
# Create "incident_id" column from "incident_url"
stage1_incidents["incident_id"] = stage1_incidents.apply(lambda x: int(x["incident_url"][len("http://www.gunviolencearchive.org/incident/"):]), axis=1)
# Delete redundant columns
del stage1_incidents["incident_url"], stage1_incidents["source_url"]
# Order columns as in incidents
cols = list(incidents)
stage1_incidents = stage1_incidents.ix[:, cols]

In [88]:
incidents = pd.concat([incidents, stage1_incidents])
incidents = incidents.sort_values(by=["date"])
incidents.drop_duplicates(subset=["incident_id"], inplace=True)
incidents = incidents.reset_index()
del incidents["index"]

# Data Cleaning

Addresses in the dataset contain abbreviations that are not applied universally, meaning that some entries might not abbreviate common words, e.g. Street, while other might do it in a different way, e.g. St, Str, or using punctuation: St., Str., etc. The following cells are looking to identify such abbreviations by computing the Levenshtein distance between addresses and observing identical addressed written in different formats.

In [89]:
def tidy_whitespaces(address):
    new_address = re.sub(' +', ' ', address)
    new_address = re.sub(' ,', ',', new_address)
    new_address = re.sub(r'^\s+', '', new_address, flags=re.UNICODE)
    new_address = re.sub(r"\s+$", "", new_address, flags=re.UNICODE)
    return new_address

In [90]:
def remove_special_characters(address):
    new_address = re.sub("\.", "", address)
    new_address = re.sub(",|\-|\/|#", " ", new_address)
    return new_address

In [91]:
def clean_address_string(address):
    if pd.isnull(address):
        return np.nan
    else:
        new_address = remove_special_characters(address)
        # Apply lookup dictionary rules
        new_address = reduce(lambda x, y: re.sub(y, lookup_dict[y], x), lookup_dict, new_address)
        new_address = tidy_whitespaces(new_address)
        return new_address

In [92]:
def strip_address(address):
    address = re.sub('([0-9]*th|[0-9]*1st|[0-9]*2nd|[0-9]*3rd)', ' ', address)
    address = re.sub('[0-9]*', '', address)
    address = re.sub('N |E |S |W |NE|NW|SE|SW', ' ', address)
    address = tidy_whitespaces(address)
    return address

In [93]:
lookup_dict = {"Pkwy": "Parkway", "Blvd": " Boulevard", "Rd": "Road", "Dr ": "Drive ", "Dr$": "Drive", "Ave ": "Avenue ", "Ave$": "Avenue", "avenues": "Avenues", "Block": "block", "Expwy": "Expressway", "Pl ": "Place ", "Pl$": "Place", "Ln": "Lane", "Northwest": "NW", "Southwest": "SW", "Northeast": "NE", "Southeast": "SE", "North-west": "NW", "South-west": "SW", "North-east": "NE", "South-east": "SE", "NorthWest": "NW", "SouthWest": "SW", "NorthEast": "NE", "SouthEast": "SE", "North-West": "NW", "South-West": "SW", "North-East": "NE", "South-East": "SE", "North ": "N ", "South ": "S ", "East ": "E ", "West ": "W ", "North$": "N", "South$": "S", "East$": "E", "West$": "W", "Av ": "Avenue ", "Av$": "Avenue", "St$": "Street"}

In [94]:
lev_incidents = incidents.sample(frac=0.01)

In [95]:
index = 0
lev_addresses = dict()
for item in list(set(lev_incidents[pd.notnull(lev_incidents["address"])]["address"].apply(clean_address_string))):
    lev_addresses[index] = strip_address(item)
    index += 1

In [96]:
# Find combinations of addresses that have between 0.95 and 0.99 similarity, disregarding numbers of streets and cardinals
for comb in itertools.combinations(lev_addresses, 2):
    similarity = lev.ratio(lev_addresses[comb[0]], lev_addresses[comb[1]])
    if similarity*100 in range(95, 100):
            print("There is a similarity of " + str(similarity) + " between the following strings:")
            print(" 1. " + lev_addresses[comb[0]])
            print(" 2. " + lev_addresses[comb[1]])

There is a similarity of 0.95 between the following strings:
 1. block of Elman Street
 2. block of Elm Street
There is a similarity of 0.95 between the following strings:
 1. block of Elman Street
 2. block of Elm Street
There is a similarity of 0.96 between the following strings:
 1. block of Martin Lu er King Boulevard
 2. block of Martin Lu er King Jr Boulevard
There is a similarity of 0.96 between the following strings:
 1. block of Maryland Avenue W
 2. block of Maryland Avenue
There is a similarity of 0.95 between the following strings:
 1. block of Gale Avenue
 2. block of Yale Avenue
There is a similarity of 0.96 between the following strings:
 1. block of Belvidere Avenue
 2. block of Belvedere Avenue
There is a similarity of 0.95 between the following strings:
 1. block of High Street
 2. block of Eigh Street
There is a similarity of 0.95 between the following strings:
 1. block of High Street
 2. block of Eigh Street


In [97]:
incidents['address'] = incidents['address'].apply(clean_address_string)

In [101]:
incidents["city_or_county"] = incidents.apply(lambda x: x["city_or_county"].split("(")[0][:-1], axis=1)

CHECKPOINT

In [53]:
participants.groupby("age").count()

,name,age_group,gender,relationship,status,type,incident_id
age,,,,,,,
0,39,90,71,1,90,90,90
1,91,219,182,0,219,219,219
2,119,332,299,14,331,333,333
3,144,411,376,24,407,411,411
4,127,364,331,15,363,364,364
5,111,311,276,14,311,312,312
6,95,290,254,13,287,290,290
7,123,310,284,8,308,310,310
8,95,288,267,11,285,288,288


In [103]:
participants[participants["age"] == 101]

,name,age,age_group,gender,relationship,status,type,incident_id
35154,James Sorrentino,101,Adult 18+,Male,None,Unharmed,Victim,139387
212794,Marion “Mamou” Hunter,101,Adult 18+,Female,None,Unharmed,Victim,557077


In [87]:
incidents[incidents["incident_id"] == 926498]

,incident_id,date,state,city_or_county,address,n_killed,n_injured,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,longitude,n_guns_involved,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,state_house_district,state_senate_district
206780,926498,2017-09-03,Ohio,Cleveland,18416 St Clair Avenue,1,0,11.0,{0: 'Unknown'},{0: 'Unknown'},"[Shot - Dead (murder, accidental, suicide), Dr...",41.572,-81.5506,1.0,{0: 209},{0: 'Adult 18+'},{0: 'Male'},NaN,NaN,{0: 'Killed'},{0: 'Victim'},10.0,21.0


According to https://www.dnainfo.com/chicago/20141211/marquette-park/marquette-park-drive-by-wounds-woman, the incidents with ID 261784 whose victim's gender is listed as "Male, female" is in fact a female.

In [78]:
participants.at[92371, "gender"] = "Female"

ages:

In [ ]:
participants.at[365231, "age"] = 31
participants.at[342096, "name"] = "Dennis L. Edwards"
participants.at[342096, "age"] = 33

In [ ]:
# name is "*"
for index in participants[participants["name"] == "*"].index:
    participants.at[index, "name"] = None
# name is an address
for index in participants[participants["name"] == "13300 Block of Frederick St"].index:
    participants.at[index, "name"] = None
    incidents.at[participants.loc[index]["incident_id"], "address"] = "13300 Block of Frederick Steet"
for index in participants[participants["name"] == "400 block of Florida Avenue"].index:
    participants.at[index, "name"] = None
# name is ":"
for index in participants[participants["name"] == ":"].index:
    participants.at[index, "name"] = None
# name is a number, age missing
for number in [20, 26, 27]:
    for index in participants[participants["name"] == str(number)].index:
        participants.at[index, "age"] = number
        participants.at[index, "name"] = None
for number in range(31, 54):
    for index in participants[participants["name"] == str(number)].index:
        participants.at[index, "age"] = number
        participants.at[index, "name"] = None
# name has an odd format
for index in participants[participants["name"] == "and Richard Lawrence"].index:
    participants.at[index, "name"] = "Richard Lawrence"
for index in participants[participants["name"] == "at least 3 homeowners"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "bMichael Mayfield"].index:
    participants.at[index, "name"] = "Michael Mayfield"
for index in participants[participants["name"] == "barbershop"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "brother"].index:
    participants.at[index, "relationship"] = "Family"
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "cashier"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "clerk"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "coach"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "cousin"].index:
    participants.at[index, "relationship"] = "Family"
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "delivery driver"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "deputy"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "deputy Daniel Rush"].index:
    participants.at[index, "name"] = "Daniel Rush"
for index in participants[participants["name"] == "dog"].index:
    participants.drop(index)
for index in participants[participants["name"] == "driver of charcoal-gray Acura"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "ex-husband"].index:   
    participants.at[index, "relationship"] = "Significant others - current or former"
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "girlfriend"].index:   
    participants.at[index, "relationship"] = "Significant others - current or former"
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "grocery store"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "his wife"].index:
    participants.at[index, "relationship"] = "Significant others - current or former"
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "homeowner"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "homeowner's friend"].index:
    participants.at[index, "relationship"] = "Friends"
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "homeowner's son"].index:
    participants.at[index, "relationship"] = "Family"
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "homeowner's wife"].index:
    participants.at[index, "relationship"] = "Family"
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "hristopher Douglas Pinguely"].index:
    participants.at[index, "name"] = "Christopher Douglas Pinguely"
for index in participants[participants["name"] == "http://kdhnews.com/news/crime/man-indicted-on-gun-theft-charge/article_fe9fde74-aee9-11e7-8c82-47881419fb6f.html"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "in car with Vinson"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "mini-mart"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "neighbor's visitor"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "not a gun vic."].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "not given"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "officer"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "officer Anthony Robinson"].index:
    participants.at[index, "name"] = "Anthony Robinson"
for index in participants[participants["name"] == "officer Garret Osilka"].index:
    participants.at[index, "name"] = "Garret Osilka"
for index in participants[participants["name"] == "officer Samuel Solorio"].index:
    participants.at[index, "name"] = "Samuel Solorio"
for index in participants[participants["name"] == "ohn Albert Angelino Jr"].index:
    participants.at[index, "name"] = "John Albert Angelino Jr"
for index in participants[participants["name"] == "ohn Lee Butler III"].index:
    participants.at[index, "name"] = "John Lee Butler III"
for index in participants[participants["name"] == "onathan Atha"].index:
    participants.at[index, "name"] ="Jonathan Atha"
for index in participants[participants["name"] == "onya B Bracy"].index:
    participants.at[index, "name"] = "Sonya B Bracy"
for index in participants[participants["name"] == "ose DeJesus Guerrero"].index:
    participants.at[index, "name"] = "Jose DeJesus Guerrero"
for index in participants[participants["name"] == "passenger"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "patient"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "petty officer of the watch"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "player"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "security guard"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "son-in-law"].index:
    participants.at[index, "relationship"] = "Family"
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "not given"].index:
    participants.at[index, "name"] = None
for index in participants[participants["name"] == "not given"].index:
    participants.at[index, "name"] = None
participants.at[20307, "name"] = None
participants.at[32192, "relationship"] = "Significant others - current or former"
participants.at[32192, "name"] = None
participants.at[54027, "name"] = None
participants.at[25399, "name"] = None
participants.at[25402, "relationship"] = "Friends"
participants.at[25402, "name"] = None
participants.at[25403, "relationship"] = "Friends"
participants.at[25403, "name"] = None
participants.at[25404, "relationship"] = "Friends"
participants.at[25404, "name"] = None
participants.at[25401, "name"] = None
participants.at[341763, "name"] = "Christopher Douglas Pinguely"
# remove the dogs from the dataset
participants.drop([20919, 24281, 24344, 34719, 38212])